# Predicting in real-time

In this notebook we will build predictive models and run them in real-time.

## 1. Set-up Spark

Copy-paste from example scripts

In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

## 2. Define the models

Some notes:
* We will define two models: VADER sentiment analyzer and logistic regression model
* For each model, we define its own `process` function since the general structure of how the predictions are obtained and appended to the dataframe changes.

In [2]:
# Import libraries
import random
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
import pyspark.sql.types as tp

### 2.1 Model 1: VADER sentiment analyzer
This is a model that we didn't even had to train, as it uses the VADER sentiment scores to score each text on how positive/negative it is.

In [3]:
# Import libraries
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to apply VADER sentiment analysis to a text string
def get_sentiment(text):
    text_str = str(text) # convert to string
    sentiment = analyzer.polarity_scores(text_str)
    if sentiment['compound'] > 0:
        return 1
    else:
        return 0

# Define a user-defined function (UDF) to apply the get_sentiment function to the review_text column of a dataframe
udf_VADER = udf(get_sentiment, tp.StringType())

# Define a function to process a Spark RDD using VADER sentiment analysis
def process_VADER(time, rdd):
    if rdd.isEmpty():
        print("rdd was empty...")
        return
    
    print("========= %s =========" % str(time))
    
    # Convert RDD to data frame
    df = spark.read.json(rdd)
    
    # Display the dataframe
    df.show()

    # Apply the udf_VADER function to the review_text column and add the 'pred' column
    df_withpreds = df.withColumn("pred", udf_VADER( struct(df.review_text) ))
    
    # Display the updated dataframe
    df_withpreds.show()


### 2.2 Model 2: Logistic regression model

General structure (data pipeline):
* Tokenize review
* 'Translate' words into vector representation
* Apply a logistic regression model to with vector representations as regressors

First we define and create the dataframe. 

In [4]:
# Import libraries
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
    
# Define the schema for the data set. 
my_schema = tp.StructType([
  tp.StructField(name= 'review_id', dataType= tp.IntegerType(), nullable= True),
  tp.StructField(name= 'app_id', dataType= tp.IntegerType(), nullable= True),
  tp.StructField(name= 'review_text', dataType= tp.StringType(), nullable= True),
  tp.StructField(name= 'label', dataType= tp.IntegerType(), nullable= True)
])

# Read the dataset into a DataFrame
my_data = spark.read.csv("C:/Users/wille/spark/MyData/review_data.csv",
                         schema=my_schema,
                         header=True)

# Probably because of the way the data set was stored as a .csv-file, the extremely long reviews are stored improperly,
# leading to missing values. Therefore, we drop these cases from the data set.
my_data = my_data.dropna()

# view the data
my_data.show(5)

# print the schema of the file
print("Schema:")
my_data.printSchema()

+---------+-------+--------------------+-----+
|review_id| app_id|         review_text|label|
+---------+-------+--------------------+-----+
|136510302|2103530|I simply love it,...|    1|
|136509602|2349550|  Gifted word: Grace|    1|
|136510134|1685730|It's pretty good!...|    1|
|136510117|1685730|I recommend it, I...|    1|
|136509657|2364130|simple point and ...|    0|
+---------+-------+--------------------+-----+
only showing top 5 rows

Schema:
root
 |-- review_id: integer (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- label: integer (nullable = true)



Then we build a logistic regression model to classify sentiment of text reviews. We do this in 4 stages: 

Stage 1: Tokenize the text of the review. This is done using a regular expression tokenizer that splits the text into individual words or tokens, removing non-word characters, such as punctuation and digits. The input column is 'review_text', and the output column is 'tokens'.

Stage 2: Remove stop words from the tokens generated in stage 1. Stop words are commonly occurring words, such as 'the' and 'and', that are typically not useful for analysis. The input column is 'tokens', and the output column is 'filtered_words'.

Stage 3: Create a word vector of size 50. This is done using the Word2Vec algorithm, which converts the filtered words from stage 2 into numerical vectors of a specified size. This stage generates a new column called 'vector'.

Stage 4: Build a Logistic Regression model to classify sentiment using the vectors generated in stage 3 as input features and the 'label' column as the target variable. This stage defines the model object, with the input features as 'vector' and the target variable as 'label'.

In [5]:
# This model will consist of several stages, elaborated further below.

# define stage 1: tokenize the tweet text    
stage_1 = RegexTokenizer(inputCol= 'review_text' , outputCol= 'tokens', pattern= '\\W')

# define stage 2: remove the stop words
stage_2 = StopWordsRemover(inputCol= 'tokens', outputCol= 'filtered_words')

# define stage 3: create a word vector of the size 50
stage_3 = Word2Vec(inputCol= 'filtered_words', outputCol= 'vector', vectorSize=50)

# define stage 4: Logistic Regression Model
model = LogisticRegression(featuresCol= 'vector', labelCol= 'label')

Next we create a pipeline, which is a way to organize multiple stages of processing that transform the data from one form to another, and then fit the pipeline to the training data.

The pipeline is created using the Pipeline function from the pyspark.ml module, and is composed of four stages, as defined in the stages list: stage_1, stage_2, stage_3, and model.Model is a LogisticRegression object that trains a binary classification model using the word vectors as features and the label column as the target variable.

Once the pipeline is defined, the fit method is used to fit the pipeline to the my_data dataset. This means that the data is transformed by passing it through the pipeline in order, and then used to train the LogisticRegression model. The resulting fitted pipeline object is stored in the pipelineFit variable.

In [6]:
# Next, we combine each of these stages together into a data pipeline...
pipeline = Pipeline(stages = [stage_1, stage_2, stage_3, model])

# and fit the model with the training data
pipelineFit = pipeline.fit(my_data)

Next, we define a function process_logistic_regression that takes in a time parameter and an RDD rdd containing review data. 

In [12]:
# Import libraries
from pyspark.sql.functions import monotonically_increasing_id, row_number, transform
from pyspark.sql.window import Window

def process_logistic_regression(time, rdd):
    # Check if RDD is empty
    if rdd.isEmpty():
        print("rdd was empty")
        return
    
    # Print time and convert to DataFrame
    print("========= %s =========" % str(time))
    df = spark.read.json(rdd)
    df.show()
    
    # Make prediction on data frame based on learned model
    out = pipelineFit.transform(df).select("prediction")
    
    # Combine the prediction (which is a data frame) with the original data frame. After searching for a long time on
    # how to do this, the only solution we found is this incredibly convoluted way (first append index column to both
    # data frames, then merge them based on this index column, then drop the index column).
    out = out.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
    df = df.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
    df_withpreds = df.join(out, on=["row_index"]).drop("row_index")
    
    # Show data frame again, alongside the predictions.
    df_withpreds.show()

## 3. Make predictions

Now we are ready to make real-time predictions 
The code sets up a streaming context (ssc) using Spark Streaming with a batch interval of 10 seconds. Then it creates a DStream (lines) by connecting to a socket and listening to incoming data on the specified host and port ("seppe.net", 7778).

Next, the foreachRDD() method is called on the DStream, which applies the specified function (process_logistic_regression()) to each RDD in the stream.

Finally, a new StreamingThread object (ssc_t) is created using the ssc object as input, and the thread is started (ssc_t.start()). This allows the stream to start receiving data and processing it using the specified function.

The last line ssc_t.stop() is redundant and not necessary for the streaming process. It simply stops the StreamingThread object created earlier.

In [13]:
# Create a streaming context with a batch interval of 10 seconds
ssc = StreamingContext(sc, 10)

# Set up a socket stream on a specified host and port
lines = ssc.socketTextStream("seppe.net", 7778)

# Specify which function to apply to each RDD in the stream
# Fill in either 'process_VADER' or 'process_logistic_regression'
lines.foreachRDD(process_logistic_regression)

# Create a thread to run the streaming context
ssc_t = StreamingThread(ssc)

# Start the streaming context
ssc_t.start()

In [11]:
# Stop the streaming context
ssc_t.stop()

----- Stopping... this may take a few seconds -----
